In [2]:
import pandas as pd 

In [35]:
rating_cols=['User Id','Movie_ID','Rating']
ratings_db=pd.read_csv("C:\\Users\\91839\\MLCourse\\ml-100k\\u.data",sep='\t',
                       names=rating_cols,usecols=range(3),encoding="ISO-8859-1")
ratings_db.head()


,User Id,Movie_ID,Rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [4]:
import numpy as np

In [5]:
comb_ratings=ratings_db.groupby('Movie_ID').agg({'Rating':[np.size,np.mean]})
comb_ratings.head()

Rating          
           size      mean
Movie_ID                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [ ]:
print(isinstance())

In [9]:
popscore_db=pd.DataFrame(comb_ratings['Rating']['size'])
popscore_db.head()
norm_popscore_db=comb_ratings_db.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))
norm_popscore_db.head()

,size
Movie_ID,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [23]:
movie_dict={}
with open(r"C:\\Users\\91839\\MLCourse\\ml-100k\\u.item",encoding="ISO-8859-1") as f:
    #the r in r"<address>" denotes reading mode 
    #the encoding "ISO-8859-1" is used for text file 
    #temp=''
    for line in f:
        #every line in the file is extracted iteratively and stored as line variable
        fields = line.rstrip('\n').split('|')
        #the line variable is split into list of fields we observe the file contains | as delimiter
        #rstrip('\n') is used to remove the trailing new line charactor 
        movieID = int(fields[0])
        name = fields[1]
        #we do not use fields 2,3,4,
        genres = fields[5:25]
        #the genres are present in string values we map them to int 
        genres = map(int,genres)
        movie_dict[movieID] = (name, np.array(list(genres)), 
                              norm_popscore_db.loc[movieID].get('size'), 
                              comb_ratings.loc[movieID].Rating.get('mean'))
        #creating entry in movieDict with movie_title as key 

In [22]:
from scipy import spatial 

In [24]:
def ComputeDistance(a,b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
ComputeDistance(movie_dict[1], movie_dict[3])


1.620926243567753

In [25]:
import operator

In [31]:
def getNeighbors(movieID, K):
    distances = []
    #array initialized for storing distance metric for all the datapoints from samplepoint
    for movie in movie_dict:
    # every movie in movie dict is getting called 
        if (movie != movieID):
        #only movies other than sample movie are being measured for distance 
            dist = ComputeDistance(movie_dict[movieID], movie_dict[movie])
            #distance computed between each movie and sample movie
            distances.append((movie, dist))
            #two elements are getting appended hence a tuple is generated 
    distances.sort(key=operator.itemgetter(1))
    #the distance is getting sorted in ascending order using the second element of the tuple i.e at index(1) 
    
    neighbors = []
    #array initialized to hold the nearest elements/movies to the sample movie
    for x in range(K):
        neighbors.append(distances[x][0])
        #K neighbors getting appended taking the movie names stored in distances array at index[0]
    return neighbors


In [32]:
K = 10
avgRating = 0
#avgRating variable is initialized 
neighbors = getNeighbors(1, K)
#neighbors array getting stored in the variable
for neighbor in neighbors:
    #every movie getting iterated
    avgRating += movie_dict[neighbor][3]
    #extracting the average rating of the neighbor movie from movie_dict and adding the sum of rating
    print (movie_dict[neighbor][0] + " " + str(movie_dict[neighbor][3]))
    
avgRating /= K
#the average Rating is taken by dividing the sum of ratings of neighbor movies with no of neighboring movies
print(avgRating)
print(movie_dict[movieID][3])

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463
3.3445905900235564
3.0


Meet John Doe (1941)                              7.347410
Return of the Jedi (1983)                         7.178172
When We Were Kings (1996)                         6.018704
Robert A. Heinlein's The Puppet Masters (1994)    5.909889
Ghost in the Shell (Kokaku kidotai) (1995)        5.607133
dtype: float64